In [12]:
import os
import json
import numpy as np
import nibabel as nib
from monai.data import create_test_image_3d
# import numpy as np
# import random
# from datetime import datetime, timedelta
# import matplotlib.pyplot as plt
# import torch
# import torch.distributed as dist
# from torch.nn.parallel import DistributedDataParallel
# from torch.cuda.amp import autocast, GradScaler
# from pathlib import Path
# from monai.transforms import Compose
# from monai.data import ThreadDataLoader, create_test_image_3d
# from monai.utils import set_determinism
# from monai.config import print_config

# print_config()

## Simulate a special dataset

It is well known that AI takes time to train. To provide the "Hello World!" experience of Auto3D in this notebook, we will simulate a small dataset and run training only for multiple epochs. Due to the nature of AI, the performance shouldn't be highly expected, but the entire pipeline will be completed within minutes!

`sim_datalist` provides the information of the simulated datasets. It lists 12 training and 2 testing images and labels. The training data are split into 3 folds. Each fold will use 8 images to train and 4 images to validate. The size of the dimension is defined by the `sim_dim`.

In [2]:
sim_datalist = {
    "training": [
        {"image": "tr_image_001.nii.gz", "label": "tr_label_001.nii.gz"},
        {"image": "tr_image_002.nii.gz", "label": "tr_label_002.nii.gz"},
        {"image": "tr_image_003.nii.gz", "label": "tr_label_003.nii.gz"},
        {"image": "tr_image_004.nii.gz", "label": "tr_label_004.nii.gz"},
        {"image": "tr_image_005.nii.gz", "label": "tr_label_005.nii.gz"},
        {"image": "tr_image_006.nii.gz", "label": "tr_label_006.nii.gz"}
    ]
}

sim_dim = (128, 128, 128)

## Generate images and labels

Now we can use MONAI `create_test_image_3d` and `nib.Nifti1Image` functions to generate the 3D simulated images under the work_dir

In [13]:
work_dir = "./helloworld_work_dir"
if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

dataroot_dir = os.path.join(work_dir, "sim_dataroot")
if not os.path.isdir(dataroot_dir):
    os.makedirs(dataroot_dir)

datalist_file = os.path.join(work_dir, "sim_datalist.json")
with open(datalist_file, "w") as f:
    json.dump(sim_datalist, f)

for d in sim_datalist["training"]:
    im, seg = create_test_image_3d(
        sim_dim[0], sim_dim[1], sim_dim[2], rad_max=10, num_seg_classes=1, random_state=np.random.RandomState(42)
    )
    image_fpath = os.path.join(dataroot_dir, d["image"])
    label_fpath = os.path.join(dataroot_dir, d["label"])
    nib.save(nib.Nifti1Image(im, affine=np.eye(4)), image_fpath)
    nib.save(nib.Nifti1Image(seg, affine=np.eye(4)), label_fpath)

print("Generated simulated images and labels.")

Generated simulated images and labels.


In [14]:
# Define the updated functions (definitions provided earlier)
from scripts.diff_model_create_training_data import diff_model_create_training_data
from scripts.diff_model_train import diff_model_train
from scripts.diff_model_infer import diff_model_infer

In [20]:
# Set up directories and configurations
env_config_path = "./configs/environment_maisi_diff_model_train.json"
model_config_path = "./configs/config_maisi_diff_model_train.json"
ckpt_filepath = None
output_prefix = "unet_3d"
amp = True
a_min = -1000
a_max = 1000
b_min = 0
b_max = 1

# Load environment and model configurations
with open(env_config_path, "r") as f:
    env_config = json.load(f)

with open(model_config_path, "r") as f:
    model_config = json.load(f)

# Set up directories based on configurations
dataroot = env_config["data_base_dir"][0]
filenames_filepath = env_config["json_data_list"][0]
output_root_embedding = env_config["output_dir"]
autoencoder_root = env_config["trained_autoencoder_path"]
list_filepath = filenames_filepath
output_dir = env_config["output_dir"]
pl_root = env_config["tfevent_path"]
ckpt_folder = env_config["model_dir"]

# Create necessary directories
os.makedirs(dataroot, exist_ok=True)
os.makedirs(output_root_embedding, exist_ok=True)
os.makedirs(autoencoder_root, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(pl_root, exist_ok=True)
os.makedirs(ckpt_folder, exist_ok=True)
os.makedirs("./predictions", exist_ok=True)

# Create pseudo filenames.txt
filenames = [f"image_{i:03d}_image.nii.gz" for i in range(10)]
with open(filenames_filepath, "w") as f:
    f.write("\n".join(filenames))

print("Created pseudo filenames.txt")

KeyError: 'data_base_dir'

In [ ]:
# Step 1: Create Training Data
print("Creating training data...")
diff_model_create_training_data(env_config, model_config_path)

In [ ]:
# Step 2: Train the Model
print("Training the model...")
diff_model_train(env_config_path, model_config_path)

In [ ]:
# Step 3: Infer using the Trained Model
print("Running inference...")
diff_model_infer(env_config_path, model_config_path, ckpt_filepath, amp)

print("Completed all steps.")